# MT2505 Computing Project Part 2 (Semester 2 2021/22)

## Preliminaries

This is the second of three Computing Projects for MT2505 Abstract Algebra. This project focuses on investigating properties of permutations using Python.

#### Submitting your project

The deadline for this part of the project is **5pm on Friday 18th March**. There are 10 marks available in this part, and it is worth 5% of your final grade in the module.

In order to submit your project, you must download it from this server **as a notebook** using `File> Download As> Notebook (.ipynb)`. You can then upload it to the relevant area on MySaint.

Your notebook must produce the correct results when run from top to bottom with a fresh Python kernel. To test this, you should click `Kernel> Restart & Run All`. You should also **validate** your project before submitting it.

The process for submitting this part of your project is therefore
1. Use `Kernel> Restart & Run All` to ensure that your code works correctly when run in sequence.
2. Click the `Validate` button in the toolbar to ensure that your project passes the included validation cells.
3. Download your project using `File> Download As> Notebook (.ipynb)`.
4. Upload this file to MySaint.

## The groups module

In this part of the project, you will use the `groups` module written by Prof J. D. Mitchell (with minor adjustments from Dr Quick). This contains functionality for working with permutations and symmetric groups.

In [138]:
from groups import *

In [139]:
G = SymmetricGroup(4)      # get the symmetric group of given degree

In [140]:
G.identity ()      # get the identity of a symmetric group

()

In [141]:
G.order()     # get the order of a symmetric group

24

In [142]:
[x for x in G]   # get a list of all the elements of a symmetric group

[(),
 (3 4),
 (2 3),
 (2 4 3),
 (2 4),
 (2 3 4),
 (1 2),
 (1 2)(3 4),
 (1 3 2),
 (1 4 3 2),
 (1 4 2),
 (1 3 4 2),
 (1 3),
 (1 4 3),
 (1 2 3),
 (1 2 4 3),
 (1 3)(2 4),
 (1 4 2 3),
 (1 4),
 (1 3 4),
 (1 4)(2 3),
 (1 3 2 4),
 (1 2 4),
 (1 2 3 4)]

In [143]:
# create permutations by specifying their disjoint cycle structure
x = Perm((1, 2), (4, 5))
y = Perm((4, 3), (2, 1))

In [144]:
x * y    # multiply/compose permutations (left to right)

(3 5 4)

In [145]:
y.degree()    # get the degree of the symmetric group to which x belongs

4

In [146]:
x.hit (3)    # apply the permutation x to the given point

3

In [147]:
x ** 2    # calculate the power of a permutation

()

In [148]:
x ** -1    # invert a permutation

(1 2)(4 5)

In [149]:
x == y     # test equality of two permutations

False

In [150]:
str(x)    # convert a permutation to a string

'(1 2)(4 5)'

In [151]:
# Check if something is a symmetric group.
# Only returns True if the input was created
# using the SymmetricGroup command.
IsSymmetricGroup (x)

False

#### Question 1

Recall that the *k-cycle* $c = (i_1\ i_2\ \ldots i_k) \in S_n$ denotes the permutation which maps $i_{j} \mapsto i_{j + 1}$ for $1 \leq j < k$, maps $i_k \mapsto i_1$, and fixes all other points. Any permutation $\sigma \in S_n$ can be written as a product of disjoint cycles.

***Write a function `cycle` which takes a `Perm` `p` and a point `i`, and returns a `tuple` containing the cycle of `p` which starts at `i`. An appropriate `Exception` should be raised if `p` is not a `Perm` or `i` is not an integer in the range `{1, 2, ..., p.degree()}`.***
<p style='text-align:right;'> <b> [2 Marks] </b> </p>

The first element in the output `tuple` should be `i` (note that this means the cycle might not have the smallest element first as is common).

For example, given `x = Perm((1, 3, 5, 6), (2, 4))`, `cycle(x, 5)` should return the `tuple` `(5, 6, 1, 3)`.

**Note:** you can create a `tuple` from a `list` using `tuple(my_list)`.


In [152]:
def cycle(p,i):
    
    #Validation checks
    if isinstance(p,Perm)!=True:
        raise TypeError("input p should be a Perm")
    if type(i)!= int:
        raise TypeError("i should be an integer")
    if i<1 or i >p.degree():
        raise ValueError("i should be in the range {1,2,...,p.degree}")
    
    #Using a list and coverting back to tuple is easier
    new_tuple = [i]
    a = p.hit(i)
    
    #keep applying the permutation until we get back to the original i
    while a!=i:
        new_tuple.append(a)
        a = p.hit(a)

    new_tuple = tuple(new_tuple)
    return new_tuple


In [153]:
# VALIDATION CELL
# Cells like this contain some validation.
# If an error is produced, you need to fix something in your code.
# If no error is produced, this is *NOT* a guarantee you have correctly answered the question.
# Do not modify these cells.
from nose.tools import *
from groups import *
if not "cycle" in globals():
    raise NotImplementedError("cycle has not been defined in Question 1")

test_perm = Perm((1, 3, 2, 4))
assert_equal(cycle(test_perm, 1), (1, 3, 2, 4))
assert_equal(cycle(test_perm, 2), (2, 4, 1, 3))

assert_raises(TypeError, cycle, "banana", 1)
assert_raises(TypeError, cycle, Perm((1, 2)), "banana")
assert_raises(ValueError, cycle, Perm((1, 2)), 5)


#### Question 2

The function `disjoint_cycles` defined below will, when given input a `Perm` `p`, return the non-trivial disjoint cycles of `p` (assuming that your function `cycle` works correctly).

In [154]:
def disjoint_cycles(p):
    if not isinstance(p, Perm):
        raise TypeError("disjoint_cycles expected a Perm, but got a ", type(p).__name__)
    seen = set()   # keep track of the points we've seen in a cycle so far
    out = set()
    for i in range(1, p.degree() + 1):
        if not i in seen:  # don't compute cycles we've already seen points in
            c = cycle(p, i)
            seen |= set(c) # update seen to contain the points in the cycle c
            if len(c) > 1: # only include the non-trivial cycles
                out.add(c)
    return out

Recall that the *order* of a permutation $\sigma$ is the smallest positive integer $k$ such that $\sigma^k = \operatorname{id}$. Running the following code cell will import a permutation with very large order. The order of this permutation is much too large to find by simply testing each integer until you find a power that works.

In [155]:
import gzip, pickle
vlp = pickle.load(gzip.open("vlp.p.gz", "r"))

***Using the function `disjoint_cycles` and any appropriate functions from the [`math` module](https://docs.python.org/3/library/math.html), compute the order of `vlp`. Store this in a variable `vlp_order`.***

<p style='text-align:right;'> <b> [2 Marks] </b> </p>

**Note:** as usual, you must show all of the computations required to justify your answer. Your notebook must execute in under 20 minutes. You might want to test your method using some of the following permutations:

`Perm((1, 2, 3), (4, 5, 6), (10, 11, 12))
Perm((1, 3), (4, 5), (6, 8, 10), (7, 9))
Perm((1, 4, 3, 2, 12, 7, 10, 5), (9, 11))
Perm((1, 8, 12, 9, 7, 5, 4, 11, 10, 2), (6, 13))
Perm((1, 9, 8, 13, 10, 5), (2, 14), (3, 6, 4, 7, 11, 12))`

The orders of these should be 3, 6, 8, 10, and 6 respectively.

In [156]:
import math
from functools import reduce # I used this to avoid errors like --- object cannot be interpeted as integer when using math.lcm()

# by theorem 8.15 ii) if the disjoint cycles have lengths ri, then the order of the permutation is equal to the 
# lowest common multiple of the lengths

p = vlp
p = disjoint_cycles(p)
lengths = []
for i in p:
    lengths.append(len(i))


vlp_order = reduce(math.lcm,lengths)  


In [157]:
# VALIDATION CELL
from nose.tools import *
from groups import *
if not "vlp_order" in globals():
    raise NotImplementedError("vlp_order has not been defined in Question 2")
if not isinstance(vlp_order, int):
    raise TypeError()


#### Question 3

Define the inversions of a permutation $\sigma \in S_n$ to be the pairs $(i, j)$ such that $1 \leq i < j \leq n$ and $i\sigma > j\sigma$. For example, $(1, 2)$ is an inversion of the permutation $\sigma = (1\ 4)(2\ 3)$ since $1 < 2$ but $1\sigma = 4 > 3 = 2\sigma$.

Denote the number of inversions of $\sigma$ by $(\sigma)\iota$.

***Write a function `number_inversions` which takes one argument, a `Perm` `p`, and returns $(p)\iota$. The function should raise an appropriate `Exception` if the argument given is not a `Perm`.***
<p style='text-align:right;'> <b> [2 Marks] </b> </p>

In [158]:

def number_inversions(p):
    
    if isinstance(p,Perm)!=True:
        raise TypeError("input p should be a Perm")
        
    inversions = 0
    
    for i in range(1,p.degree()+1):
        
        #second loop starts at i+1 as i<j as in definition
        for x in range(i+1,p.degree()+1):
            
            #checking if is>js
            if p.hit(i)>p.hit(x):
                inversions +=1
    return inversions
        
                    

In [159]:
# VALIDATION CELL
from nose.tools import *
from groups import *
if not "number_inversions" in globals():
    raise NotImplementedError("number_inversions has not been defined in Question 3")

assert_equal(number_inversions(Perm((1, 4), (2, 3))), 6)
    

#### Question 4

Given a permutation $\sigma$, write $(\sigma)\nu$ for the number of cycles of even length in its disjoint cycle representation.

***Use Python to show that for all $\sigma \in S_7$, $(\sigma)\nu \equiv (\sigma)\iota \mod 2$, where $\iota$ is as defined in Question 3.***

<p style='text-align:right;'> <b> [2 Marks] </b> </p>

In [160]:
S = SymmetricGroup(7)
S = [x for x in S]

def proof(S):
    
    #check every permutation in S7
    for i in S:
        sigma_v = 0
        inversions = number_inversions(i)
        disjoint_i = disjoint_cycles(i)

        #check every cycle of even length in the permutation
        for x in disjoint_i:
            if len(x)%2 ==0:
                sigma_v +=1

        #if the condition does not hold , return false
        if (sigma_v - inversions)%2 !=0:
            return False
    return True

proof(S)


True

#### Question 5

***Find any permutation $\sigma$ such that the order of $\sigma$ is precisely 8 times the order of `vlp`. Store this permutation as `bigger_perm`. You must justify your answer.***

<p style='text-align:right;'> <b> [2 Marks] </b> </p>

*Hint:* compute $(\text{vlp})\nu$.

In [161]:
vlp = pickle.load(gzip.open("vlp.p.gz", "r"))
vlp.degree() # is equal to 4999


4999

In [162]:
vlp_dis = disjoint_cycles(vlp)
vlp_v = 0
for i in vlp_dis:
    if len(i)%2==0:
        vlp_v +=1
vlp_v # is equal to zero so there are no disjoint cycles of even length.

0

By theorem 8.15 if the disjoint cycles have lengths ri, then the order of the permutation is equal to the lowest common multiple of the lengths . So adding a disjoint cycle of length 8 will multiply the order by 8, since it will
be the only cycle of even length.

In [163]:

vlp_dis.add((5000,5001,5002,5003,5004,5005,5006,5007))

#Testing solution
lengths = []
for i in vlp_dis:
    lengths.append(len(i))


sigma_order = reduce(math.lcm,lengths)
original_vlp_order = 94440262354631016982509501739431515720297024775 # This is from the calculation in question 2

sigma_order/original_vlp_order # returns 8, confirming the order of the permutation is 8 times the order of vlp.

8.0

In [164]:
#Storing permutation bigger_perm
y = Perm((5000,5001,5002,5003,5004,5005,5006,5007))
bigger_perm = vlp*y

In [165]:
# VALIDATION CELL
from nose.tools import *
from groups import *
if not "bigger_perm" in globals():
    raise NotImplementedError("bigger_perm has not been defined in Question 5")
if not isinstance(bigger_perm, Perm):
    raise TypeError()